### Importing the Needed Modules

In [0]:
import sys
import os

# Absolute path to the repo root
PROJECT_ROOT = "/Workspace/Users/thiruvengadamk16@gmail.com/Retail-And-Ecommerce-Analytics-Platform"

# Add repo root to PYTHONPATH (only once)
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

# Debug checks (safe to remove later)
print("Current working directory:", os.getcwd())
print("Repo root added to path:", PROJECT_ROOT in sys.path)
from src.paths import *

In [0]:
import os
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable
import sys
sys.path.append("/Workspace/Users/thiruvengadamk16@gmail.com/Retail-And-Ecommerce-Analytics-Platform")

from src.paths import BRONZE_STORES_PATH, SILVER_STORES_PATH
from src.schema_definitions import SILVER_STORES_SCHEMA

### Querying the Bronze Stores Table

In [0]:
stores_bronze_df = spark.read.table(BRONZE_STORES_PATH)
stores_bronze_df.limit(5).display()

### Silver Stores Schema Reference

In [0]:
SILVER_STORES_SCHEMA

### Schema Enforecement and Column Name Standardizing

In [0]:
stores_silver_df = stores_bronze_df.select(
    col("Store ID").cast(IntegerType()).alias("store_id"),
    col("Country").cast(StringType()).alias("country"),
    col("City").cast(StringType()).alias("city"),
    col("Store Name").cast(StringType()).alias("store_name"),
    col("Number of Employees").cast(IntegerType()).alias("number_of_employees"),
    col("ZIP Code").cast(StringType()).alias("zip_code"),
    col("Latitude").cast(DoubleType()).alias("latitude"),
    col("Longitude").cast(DoubleType()).alias("longitude"),
    col("ingestion_ts"),
    col("_source_file")
)


### Cleaning Empty spaces and Formatting values

In [0]:
stores_silver_df = (
    stores_silver_df.withColumn("country", upper(trim(col("country"))))
    .withColumn("city", initcap(trim(col("city"))))
    .withColumn("store_name", initcap(trim(col("store_name"))))
    .withColumn("zip_code", trim(col("zip_code")))
)

In [0]:
stores_silver_df.limit(5).display()

### Dropping Duplicate Rows

In [0]:
print(f"Before deduplicate count : {stores_silver_df.count()}")
stores_silver_df = stores_silver_df.dropDuplicates(["store_id", "country"])
print(f"After deduplicate count : {stores_silver_df.count()}")

### Filtering out null store_id, country, and store_name rows

In [0]:
stores_silver_df = stores_silver_df.filter('(store_id is not null) and (country is not null) and (store_name is not null)')

### validating the primary Key

In [0]:
stores_silver_df.groupBy("store_id").agg(count('*').alias("count")).filter('count > 1').display()

### Null Check

In [0]:
null_counts = stores_silver_df.select([sum(col(c).isNull().cast(IntegerType())).alias(c) for c in stores_silver_df.columns])
display(null_counts)

### Schema Enforcement Check

In [0]:
expected_cols = set(SILVER_STORES_SCHEMA.keys())
incoming_cols = set(stores_silver_df.columns)

unknown_cols = incoming_cols - expected_cols - {
    "ingestion_ts", "_source_file"
}

print("Unknown columns in Bronze:", unknown_cols)


### Creating or Updating Silver Stores Table

In [0]:
if not spark.catalog.tableExists(SILVER_STORES_PATH):
    (
        stores_silver_df.write
        .format("delta")
        .mode("overwrite")
        .saveAsTable(SILVER_STORES_PATH)
    )
else:
    stores_silver_tbl = DeltaTable.forName(spark, SILVER_STORES_PATH)

    stores_silver_tbl.alias("tgt").merge(
        stores_silver_df.alias("src"),
        "tgt.store_id = src.store_id "
        "AND tgt.country = src.country"
    ).whenMatchedUpdateAll() \
    .whenNotMatchedInsertAll() \
    .execute()

In [0]:
spark.read.table(SILVER_STORES_PATH).limit(5).display()

In [0]:
spark.read.table(SILVER_STORES_PATH).count()